In [ ]:
import os
import pandas as pd

# Specify the directory where the files are located
directory = "/kaggle/input/vollmer-csv/vollmer_s0"  # Replace with the actual directory path

# Create an empty list to store the data from the CSV files
df = pd.DataFrame()
data = []
# Loop through the desired file numbers (1 to 13) with two digits
for file_number in range(1, 14):
    # Use string formatting to ensure two-digit file numbers
    file_name = f"{file_number:02d}.csv"
    #print(file_name)
    file_path = directory + file_name
    #print(file_path)
    data = pd.read_csv(file_path)
    third_column = data.iloc[:, 1] 
    #print(third_column.shape)
    df = pd.concat([df, third_column], axis=1, ignore_index=True)
    #print(df.shape)

# Now, data_list contains the data from the 13 specific CSV files with two-digit file numbers.

print(df.shape)


In [ ]:
import tensorflow as tf

# Check for GPU and set memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPUs are available and memory growth is set")
    except RuntimeError as e:
        print(e)

In [ ]:
import os
import pandas as pd

# Specify the directory where the files are located
directory = "/kaggle/input/vollmer-csv/vollmer_labels_s0"  # Replace with the actual directory path

# Create an empty list to store the data from the CSV files
cut_points = pd.DataFrame()

# Loop through the desired file numbers (1 to 13) with two digits
for file_number in range(1, 14):
    # Use string formatting to ensure two-digit file numbers
    file_name = f"{file_number:02d}.csv"
    #print(file_name)
    file_path = directory + file_name
    #print(file_path)
    cut = pd.read_csv(file_path)
    #print(cut)
    selected_columns = cut[['FAROS_Marker/Rest', 'FAROS_Marker/Walking', 'FAROS_Marker/2-Back','Manual/Running']]

    #print(selected_columns3)
    cut_points = pd.concat([cut_points, selected_columns], axis=0, ignore_index=True)

# Now, data_list contains the data from the 13 specific CSV files with two-digit file numbers.

print(cut_points.shape)


In [ ]:
ecg = df[0]
    #print(ecg)
cut_point = cut_points.iloc[0]
cut_point1 = cut_point[0]
ecg = ecg.to_frame()
ecg.reset_index(drop=True, inplace=True)
#print(ecg.T)
rest = ecg.iloc[cut_point1:cut_point1+76800]
rest.reset_index(drop=True, inplace=True)
#print(rest.T)

In [ ]:
import numpy as np
import pandas as pd
result = pd.DataFrame()
labels = []
for i in range(13):
    ecg = df[i]
    #print(ecg)
    cut_point = cut_points.iloc[i]
    cut_point1 = cut_point[0]
    ecg = ecg.to_frame()
 
    #print(cut_point[1])
    rest = ecg.iloc[cut_point1:cut_point1+76800]
    rest.reset_index(drop=True, inplace=True)
    result = pd.concat([result, rest], axis=1, ignore_index=True)

    #walking = ecg.iloc[cut_point[1]:cut_point[1]+76800]
    #walking.reset_index(drop=True, inplace=True)
    #result = pd.concat([result, walking], axis=1, ignore_index=True)

    stress = ecg.iloc[cut_point[2]:cut_point[2]+76800]
    stress.reset_index(drop=True, inplace=True)
    result = pd.concat([result, stress], axis=1, ignore_index=True)

    #running = ecg.iloc[cut_point[3]:cut_point[3]+76800]
    #running.reset_index(drop=True, inplace=True)
    #result = pd.concat([result, running], axis=1, ignore_index=True)
    
    label = [0,1]
    labels = labels+label

#print(result)
print(labels)

In [ ]:
print(result.shape)

In [ ]:
import pandas as pd
import numpy as np
original_data = result
# Assuming 'original_data' is your (76800, 26) DataFrame
rows, columns = original_data.shape

# Step 1: Initialize an empty list to store segments for each column
segment_list = []

# Step 2: Loop through each column to create 30 segments for each
segments_per_column = 30
data_points_per_segment = rows // segments_per_column  # 76800 / 30 = 2560

for col in range(columns):
    # Get the data points for the current column
    column_data = original_data.iloc[:, col].values
    
    # Split this column's data into 30 segments, each of length 2560
    segmented_column_data = column_data.reshape(segments_per_column, data_points_per_segment).T
    segment_list.append(segmented_column_data)

# Step 3: Stack all segmented columns horizontally to create the final (2560, 780) DataFrame
final_df = pd.DataFrame(np.hstack(segment_list))

# Step 4: Create the alternating 0s and 1s array for (780, 1) shape
labels = np.tile([0, 1], columns // 2) if columns % 2 == 0 else np.tile([0, 1], columns // 2 + 1)[:columns]
y_df = pd.DataFrame(np.repeat(labels, segments_per_column).reshape(-1, 1))

# Verify final shapes
print("Final DataFrame shape:", final_df.shape)  # Should be (2560, 780)
print("Label DataFrame shape:", y_df)#.shape)      # Should be (780, 1)


In [ ]:
import matplotlib.pyplot as plt

# Select the data for the first subject
subject_data = result.iloc[:200, 1]  # Assuming the first column represents the first subject

# Create a time axis for the plot
time_axis = range(200)

# Plot the data
plt.plot(time_axis, subject_data)
plt.xlabel('Time')
plt.ylabel('Voltage(mV)')
plt.title('Plot of First 1000 Data Points - Subject 1')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Select the data for the first subject
#wesad_x = pd.DataFrame(concatenated_results.T)
subject_data = final_df.iloc[:200, 30]  # Assuming the first column represents the first subject

# Create a time axis for the plot
time_axis = range(200)

# Plot the data
plt.plot(time_axis, subject_data)
plt.xlabel('Time')
plt.ylabel('Voltage(mV)')
plt.title('Plot of First 1000 Data Points - Subject 1')
plt.show()

In [ ]:
# filtring order = 2 
import numpy as np

#features2 = concatenated_results.values
# Convert the Pandas DataFrame 'features' to a NumPy array

filtered_data = final_df.T
print(filtered_data.shape)

In [ ]:
import numpy as np
repeated_y = y_df

# Verify the shape of the resulting array
print(repeated_y.shape) 

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Assuming filtered_data and repeated_y (as y) are already defined
# Convert repeated_y to a NumPy array before splitting
repeated_y = np.array(repeated_y)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Add, Dense, Input, Conv1D, MaxPooling1D, BatchNormalization, Dropout, GlobalMaxPooling1D, Concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import numpy as np
from tensorflow.keras.layers import Bidirectional, LSTM
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization, Add, Concatenate, ZeroPadding1D
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, BatchNormalization, SeparableConv1D, Bidirectional, GRU, LSTM, Dense, Add, Concatenate, GlobalAveragePooling1D, Reshape, Attention
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import GlobalAveragePooling1D, Dense, Multiply
from tensorflow.keras.layers import SeparableConv1D, AveragePooling1D, BatchNormalization, Concatenate

def se_block(input_layer, reduction_ratio=4):
    filters = input_layer.shape[-1]
    se = GlobalAveragePooling1D()(input_layer)
    se = Dense(filters // reduction_ratio, activation='relu')(se)
    se = Dense(filters, activation='sigmoid')(se)
    return Multiply()([input_layer, se])

class PositionalEncoding(Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.position = position
        self.d_model = d_model

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, inputs):
        seq_length = tf.shape(inputs)[1]
        angles = self.get_angles(np.arange(self.position)[:, np.newaxis],
                                 np.arange(self.d_model)[np.newaxis, :],
                                 self.d_model)

        # Apply sine to even indices in the array; 2i
        angles[:, 0::2] = np.sin(angles[:, 0::2])

        # Apply cosine to odd indices in the array; 2i+1
        angles[:, 1::2] = np.cos(angles[:, 1::2])

        pos_encoding = angles[np.newaxis, ...]
        pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)

        # Scale the input (optional, closer to original implementation)
        inputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        return inputs + pos_encoding[:, :seq_length, :]



def shared_cnn(input_layer, filters1, kernel_size1, filters2, kernel_size2, filters3, kernel_size3):
    # First CNN block
    conv1 = Conv1D(filters=filters1, kernel_size=kernel_size1, activation='relu')(input_layer)
    # Apply SE block after conv2
    conv1 = se_block(conv1)
    maxpool1 = AveragePooling1D(pool_size=2)(conv1)
    batch_norm1 = BatchNormalization()(maxpool1)
    
    # Second CNN block
    conv2 = Conv1D(filters=filters2, kernel_size=kernel_size2, activation='relu')(batch_norm1)
    # Apply SE block after conv2
    conv2 = se_block(conv2)
    maxpool2 = AveragePooling1D(pool_size=2)(conv2)
    batch_norm2 = BatchNormalization()(maxpool2)
 
    conv3 = Conv1D(filters=filters3, kernel_size=kernel_size3, activation='relu')(batch_norm2)
    #conv3 = se_block(conv3)
    maxpool3 = AveragePooling1D(pool_size=2)(conv3)
    batch_norm3 = BatchNormalization()(maxpool3)
   
    return batch_norm3


def hybrid_depthwise_cnn(input_layer, filters1, kernel_size1, filters2, kernel_size2, filters3, kernel_size3):
    conv1 = SeparableConv1D(filters=filters1, kernel_size=kernel_size1, activation='relu')(input_layer)
    conv1 = se_block(conv1)
    maxpool1 = AveragePooling1D(pool_size=2)(conv1)
    batch_norm1 = BatchNormalization()(maxpool1)

    conv2 = SeparableConv1D(filters=filters2, kernel_size=kernel_size2, activation='relu')(batch_norm1)
    conv2 = se_block(conv2)
    maxpool2 = AveragePooling1D(pool_size=2)(conv2)
    batch_norm2 = BatchNormalization()(maxpool2)

    conv3 = SeparableConv1D(filters=filters3, kernel_size=kernel_size3, activation='relu')(batch_norm2)
    #conv3 = se_block(conv3)
    maxpool3 = AveragePooling1D(pool_size=2)(conv3)
    batch_norm3 = BatchNormalization()(maxpool3)
   
    return batch_norm3


def transformer_encoder(input_layer, num_heads, units):
    # Layer normalization before self-attention (optional change for Pre-Norm)
    norm_input = LayerNormalization()(input_layer)
    
    # Self-Attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=units)(norm_input, norm_input)
    attention_output = Add()([input_layer, attention_output])  # Residual connection

    # Layer normalization before FFN (optional change for Pre-Norm)
    norm_attention = LayerNormalization()(attention_output)
    
    # Feed-forward neural network with expansion
    ffn = Dense(4 * units, activation='relu')(norm_attention)  # Expansion
    ffn_output = Dense(units)(ffn)  # Reduce back to `units`
    ffn_output = Add()([attention_output, ffn_output])  # Residual connection
    transformer_output = LayerNormalization()(ffn_output)

    return transformer_output




In [ ]:
k_num = 13

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Convert `repeated_y` to a NumPy array if it's a DataFrame or Series
from tensorflow.keras.layers import MultiHeadAttention, Flatten

if not isinstance(repeated_y, np.ndarray):
    repeated_y = repeated_y.to_numpy()
#print(repeated_y)
# Initialize KFold with 8 splits
kf = KFold(n_splits=(k_num), shuffle=False, random_state=None)

# Initialize lists to store the history and results of each fold
all_fold_histories = []
all_fold_accuracies = []

fold = 1

for train_index, test_index in kf.split(filtered_data):
    #if fold not in [2, 3, 6, 7, 13]:
        #fold += 1
         #continue

    tf.keras.backend.clear_session()
    print(f"\nTraining on Fold 2nd {fold}...")
    print(test_index)
    filtered = filtered_data.T
    # Split the data
    X_train, X_test = filtered[train_index].T, filtered[test_index].T
    y_train, y_test = repeated_y[train_index], repeated_y[test_index]

    print(X_train.shape)
    print(X_test.shape)
    y_train = y_train.ravel()  # Ensure y_train is a 1D array
    y_test = y_test.ravel()    # Ensure y_test is a 1D array
        
    print(y_train.shape)
    print(y_test)
    
    # Check the shape of X_train and X_test
    print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")

    # Standard scaling
    scaling2 = StandardScaler()
    X_train = scaling2.fit_transform(X_train)
    X_test = scaling2.transform(X_test)  # Now X_test should match X_train's feature count
    input_layer = Input(shape=(2560, 1))
    shared_cnn_output2 = shared_cnn(input_layer, filters1=32, kernel_size1=14, filters2=64, kernel_size2=8, filters3=128, kernel_size3=5)
# Apply hybrid CNN
    hybrid_cnn_output2 = hybrid_depthwise_cnn(input_layer, filters1=32, kernel_size1=14, filters2=64, kernel_size2=8, filters3=128, kernel_size3=5)
# Concatenate CNN outputs 
    combined_output2 = Add()([shared_cnn_output2, hybrid_cnn_output2])
    combined_output2 = se_block(combined_output2)
# Apply a Bidirectional LSTM layer after each CNN output
    bilstm_output2 = Bidirectional(LSTM(units=64, return_sequences=True))(combined_output2)                                                                      
# Apply Positional Encoding after LSTM outputs
    pos_encoder2 = PositionalEncoding(position=2560, d_model=128)(bilstm_output2)   # d_model matches LSTM output size
# Apply Transformer Encoder to each Positional Encoding output
    transformer_output2 = transformer_encoder(pos_encoder2, num_heads=4, units=128)
# Apply a second Transformer Encoder layer if needed
    transformer_output2_2 = transformer_encoder(transformer_output2, num_heads=2, units=128)

# Apply a second Transformer Encoder layer if needed
#transformer_output1_2 = transformer_encoder(transformer_output1_1, num_heads=2, units=256)
    transformer_output2_3 = transformer_encoder(transformer_output2_2, num_heads=2, units=128)
    transformer_output2_4 = transformer_encoder(transformer_output2_3, num_heads=2, units=128)
    transformer_output2_5 = transformer_encoder(transformer_output2_4, num_heads=2, units=128)
    transformer_output2_6 = transformer_encoder(transformer_output2_5, num_heads=2, units=128)
    transformer_output2_7 = transformer_encoder(transformer_output2_6, num_heads=2, units=128)
    transformer_output2_8 = transformer_encoder(transformer_output2_7, num_heads=2, units=128)
    transformer_output2_9 = transformer_encoder(transformer_output2_8, num_heads=2, units=128)

# Residual connection
    residual_output2 = Add()([transformer_output2_5, bilstm_output2])
# Apply Global Max Pooling to flatten the outputs
    pooled_output2 = GlobalAveragePooling1D()(residual_output2)
# Concatenate the outputs of both 
# Dense layers
    dense1 = Dense(units=256, activation='relu', kernel_regularizer=l2(0.01))(pooled_output2)
    batch_norm4 = BatchNormalization()(dense1)
    drop1 = Dropout(0.4)(batch_norm4)
    dense2 = Dense(units=128, activation='relu', kernel_regularizer=l2(0.01))(drop1)
    batch_norm5 = BatchNormalization()(dense2)
    drop2 = Dropout(0.4)(batch_norm5)
# Output layer for binary classification
    output = Dense(units=1, activation='sigmoid')(drop2)
# Build and compile the model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    es = EarlyStopping(monitor='val_loss', mode='min', patience=80)
    
    # Check shapes
    print(f"Fold {fold} - X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

    # Define a unique filename for each fold
    checkpoint_filename = f'best_model_fold_3nd{fold}.keras'
    mc = ModelCheckpoint(checkpoint_filename, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

    # Train the model with early stopping and a unique checkpoint for each fold
    try:
        history = model.fit(X_train, y_train, epochs=150, batch_size=32,
                            validation_data=(X_test, y_test), callbacks=[es, mc])

        # Save the history and validation accuracy for this fold
        all_fold_histories.append(history.history)
        all_fold_accuracies.append(max(history.history['val_accuracy']))
        
        print(f"Fold {fold} - Validation accuracy: {all_fold_accuracies[-1]}")
    except Exception as e:
        print(f"Error during training Fold {fold}: {e}")
    
    tf.keras.backend.clear_session()
    fold += 1

# Print or analyze the results
print("\nCross-validation accuracies for each fold:", all_fold_accuracies)
print("Average cross-validation accuracy:", np.mean(all_fold_accuracies))


In [ ]:
from keras import layers
import keras
import numpy as np
import tensorflow as tf

@keras.saving.register_keras_serializable(package="Custom", name="PositionalEncoding")
class PositionalEncoding(layers.Layer):
    def __init__(self, position, d_model, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.position = position
        self.d_model = d_model

    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
        return pos * angle_rates

    def call(self, inputs):
        seq_length = tf.shape(inputs)[1]
        angles = self.get_angles(np.arange(self.position)[:, np.newaxis],
                                 np.arange(self.d_model)[np.newaxis, :],
                                 self.d_model)

        # Apply sine to even indices in the array; 2i
        angles[:, 0::2] = np.sin(angles[:, 0::2])

        # Apply cosine to odd indices in the array; 2i+1
        angles[:, 1::2] = np.cos(angles[:, 1::2])

        pos_encoding = angles[np.newaxis, ...]
        pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)

        # Scale the input (optional, closer to original implementation)
        inputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        return inputs + pos_encoding[:, :seq_length, :]

    def get_config(self):
        config = super().get_config()
        config.update({
            "position": self.position,
            "d_model": self.d_model,
        })
        return config


In [ ]:
print(filtered_data.shape)


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import cohen_kappa_score

kappa_scores = []
# Initialize KFold with 4 splits
kf = KFold(n_splits=k_num, shuffle=False, random_state=None)
print(filtered_data.shape)
fold = 1
for train_index, test_index in kf.split(filtered_data):
    #if fold not in [2, 3, 6, 7, 13]:
        #fold += 1
        #continue
    print(f"\nResult on Fold {fold}...")

    # Split the data
    X_train1, X_test1 = filtered_data.T[train_index], filtered_data.T[test_index]
    y_train, y_test = repeated_y[train_index], repeated_y[test_index]

    # Standard scaling
    scaling = StandardScaler()
    X_train = scaling.fit_transform(X_train1.T)
    X_test = scaling.transform(X_test1.T)

    # Reshape y_train and y_test to 1D arrays for proper indexing
    y_train = y_train.ravel()
    y_test = y_test.ravel()

    #print(fold)    # Load the best model for this fold to evaluate
    model_fold = load_model(f'best_model_fold_3nd{fold}.keras', custom_objects={"PositionalEncoding": PositionalEncoding})

        # Get the predicted probabilities for the test set
    print(X_test.shape)

    print(X_train.shape)

    
    y_pred_probs = model_fold.predict(X_test)

        # Convert predicted probabilities to binary class labels (0 or 1)
    y_pred_classes = (y_pred_probs > 0.5).astype(int).flatten()

        # Generate the confusion matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    print("Confusion Matrix for Fold {}:\n".format(fold), cm)

        # Print the classification report for precision, recall, f1-score
    report = classification_report(y_test, y_pred_classes, target_names=['Class 0', 'Class 1'])
    print("\nClassification Report for Fold {}:\n".format(fold), report)

    # Compute ROC curve and ROC area
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
    roc_auc = auc(fpr, tpr)

    # Plotting the ROC curve
    plt.figure(figsize=(10, 6))
    plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC Curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')  # Diagonal line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver Operating Characteristic for Fold {fold + 1}')
    plt.legend(loc='lower right')
    plt.grid()
    plt.show() 
    
    kappa = cohen_kappa_score(y_test, y_pred_classes)
    kappa_scores.append(kappa)
    print(f"Kappa for this fold: {kappa}")
    # Optionally print AUC value
    print(f'Fold {fold + 1} - AUC: {roc_auc:.2f}')
    
    fold += 1

# Print or analyze the results
print("\nCross-validation accuracies for each fold:", all_fold_accuracies)
print("Average cross-validation accuracy:", np.mean(all_fold_accuracies))


In [ ]:
print("Number of folds in all_fold_histories:", len(all_fold_histories))


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

# Assuming all_fold_histories contains the history from each fold
for fold in range(1, 13+1):  # Adjust the range based on your number of folds

    history = all_fold_histories[fold - 1]  # Get the history for the current fold

    # Plotting accuracy
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(history['accuracy'], label='Training Accuracy')
    plt.plot(history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Fold {fold} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plotting loss
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'], label='Training Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title(f'Fold {fold} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.suptitle(f'Fold {fold} Performance')
    plt.show()
